In [37]:
from dotenv import load_dotenv
load_dotenv()


False

In [38]:
%pip install psycopg[binary]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
import os
import psycopg

# Load DATABASE_URL from environment variable
os.chdir("..")
print("Current working directory:", os.getcwd())
print("Directory contents:", os.listdir())

DATABASE_URL = os.getenv("DATABASE_URL")

if not DATABASE_URL:
    raise RuntimeError("DATABASE_URL environment variable not set. Please set it to your Neon database connection string.")

# Connect to the Neon Postgres database
try:
    conn = psycopg.connect(DATABASE_URL)
    print("✅ Successfully connected to Neon database!")
except Exception as e:
    print("❌ Failed to connect to Neon database:", e)
    conn = None


Current working directory: c:\
Directory contents: ['$Recycle.Bin', '$WinREAgent', '.GamingRoot', 'AMD', 'bootmgr', 'BOOTNXT', 'Config.Msi', 'Documents and Settings', 'DumpStack.log', 'DumpStack.log.tmp', 'hiberfil.sys', 'inetpub', 'Intel', 'mylog.log', 'pagefile.sys', 'PerfLogs', 'Program Files', 'Program Files (x86)', 'ProgramData', 'Recovery', 'RHDSetup.log', 'Riot Games', 'Ruby33-x64', 'swapfile.sys', 'System Volume Information', 'Users', 'Windows', 'XboxGames']
✅ Successfully connected to Neon database!


In [40]:
if conn:
    with conn.cursor() as cur:
        # Get column names
        cur.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'silver_classes' AND table_schema = 'public';")
        columns = [row[0] for row in cur.fetchall()]

        # For each column, count non-null values grouped by 'source'
        print("Non-null value counts for each column, grouped by 'source' (excluding sources containing '%test%'):")
        for col in columns:
            if col == 'source':
                continue  # skip grouping column itself
            query = f"""
                SELECT source, COUNT({col}) AS non_null_count
                FROM public.silver_classes
                WHERE {col} IS NOT NULL
                  AND source NOT ILIKE '%test%'
                GROUP BY source
                ORDER BY source;
            """
            cur.execute(query)
            results = cur.fetchall()
            print(f"\nColumn: {col}")
            if results:
                for source, count in results:
                    print(f"  Source: {source} | Non-null count: {count}")
            else:
                print("  No non-null values found for this column.")


Non-null value counts for each column, grouped by 'source' (excluding sources containing '%test%'):

Column: class_id
  Source: coolcharm | Non-null count: 1610
  Source: koepel | Non-null count: 401
  Source: rite | Non-null count: 4291
  Source: rowreformer | Non-null count: 4981

Column: class_name
  Source: coolcharm | Non-null count: 1610
  Source: koepel | Non-null count: 401
  Source: rite | Non-null count: 4291
  Source: rowreformer | Non-null count: 4981

Column: instructor
  Source: coolcharm | Non-null count: 46
  Source: koepel | Non-null count: 387
  Source: rite | Non-null count: 4291
  Source: rowreformer | Non-null count: 4928

Column: location
  Source: coolcharm | Non-null count: 1610
  Source: koepel | Non-null count: 401
  Source: rite | Non-null count: 4291
  Source: rowreformer | Non-null count: 4981

Column: start_ts
  Source: coolcharm | Non-null count: 1607
  Source: koepel | Non-null count: 400
  Source: rite | Non-null count: 4109
  Source: rowreformer | Non-

In [41]:
if conn:
    with conn.cursor() as cur:
        # Get all unique sources (excluding test sources)
        cur.execute("""
            SELECT DISTINCT source
            FROM public.silver_classes
            WHERE source NOT ILIKE '%test%'
            ORDER BY source;
        """)
        sources = [row[0] for row in cur.fetchall()]
        print("\nShowing 3 random rows for each source (excluding sources containing '%test%'):\n")
        for source in sources:
            print(f"Source: {source}")
            cur.execute(f"""
                SELECT *
                FROM public.silver_classes
                WHERE source = %s
                ORDER BY random()
                LIMIT 3;
            """, (source,))
            rows = cur.fetchall()
            if rows:
                # Print column headers
                colnames = [desc[0] for desc in cur.description]
                print(" | ".join(colnames))
                for row in rows:
                    print(" | ".join(str(val) if val is not None else "" for val in row))
            else:
                print("  No rows found for this source.")
            print("-" * 60)



Showing 3 random rows for each source (excluding sources containing '%test%'):

Source: coolcharm
class_id | source | class_name | instructor | location | start_ts | end_ts | capacity | spots_available | status | url | first_seen_at | last_updated_at | last_scraped_at | is_cancelled | is_past | source_run_id | source_snapshot_id | raw_data
coolcharm:381712782516 | coolcharm | Tower mixed level (All levels) |  | Genk | 2025-09-09 18:30:00+00:00 | 2025-09-09 19:25:00+00:00 | 5 | 1 | Book |  | 2025-09-01 11:40:12.187097+00:00 | 2025-09-01 11:40:12.187097+00:00 | 2025-09-01 11:38:58.025561+00:00 | False | False | 65569fca-1565-4ce2-bd9b-267f84bf45c0 | 297927 | {'date': '09/09/2025', 'time': '18:30 - 19:25', 'location': 'Genk', 'class_name': 'Tower mixed level (All levels)', 'availability': '1 / 5', 'booking_status': 'Book'}
coolcharm:725247609879 | coolcharm | Mat + Tower (All Levels) |  | Antwerp | 2025-09-16 10:00:00+00:00 | 2025-09-16 10:50:00+00:00 |  |  | Book |  | 2025-09-01 11:18:3

In [42]:
if conn:
    with conn.cursor() as cur:
        print("\nChecking for duplicate combinations of (source, class_name, instructor, location, start_ts, end_ts):\n")
        cur.execute("""
            SELECT 
                source, class_name, instructor, location, start_ts, end_ts, COUNT(*) as dup_count
            FROM public.silver_classes
            GROUP BY source, class_name, instructor, location, start_ts, end_ts
            HAVING COUNT(*) > 1
            ORDER BY dup_count DESC, source, class_name, start_ts;
        """)
        duplicates = cur.fetchall()
        if duplicates:
            print(f"Found {len(duplicates)} duplicate combination(s):")
            for row in duplicates:
                print(f"Source: {row[0]} | Class Name: {row[1]} | Instructor: {row[2]} | Location: {row[3]} | Start: {row[4]} | End: {row[5]} | Count: {row[6]}")
        else:
            print("No duplicate combinations found.")



Checking for duplicate combinations of (source, class_name, instructor, location, start_ts, end_ts):

Found 1607 duplicate combination(s):
Source: rowreformer | Class Name: ROW Class | Instructor: None | Location: ROW Studio | Start: None | End: None | Count: 51
Source: rowreformer | Class Name: REFORM | Instructor: Kim | Location: Reformer studio | Start: None | End: None | Count: 44
Source: rowreformer | Class Name: REFORM | Instructor: Viktoria | Location: Reformer studio | Start: None | End: None | Count: 31
Source: rowreformer | Class Name: REFORM | Instructor: Elyne | Location: Reformer studio | Start: None | End: None | Count: 24
Source: rowreformer | Class Name: REFORM | Instructor: Rachael | Location: Reformer studio | Start: None | End: None | Count: 16
Source: rowreformer | Class Name: REFORM | Instructor: Viktoria (Substitute) | Location: Reformer studio | Start: 2025-06-06 08:00:00+00:00 | End: 2025-06-06 08:50:00+00:00 | Count: 15
Source: rowreformer | Class Name: REFORM

In [43]:
if conn:
    with conn.cursor() as cur:
        print("\nMost recent class from each source (where start_ts and end_ts are not null):\n")
        cur.execute("""
            SELECT DISTINCT ON (source)
                source, class_name, instructor, location,capacity, spots_available, start_ts, end_ts, status, url
            FROM public.silver_classes
            WHERE start_ts IS NOT NULL AND end_ts IS NOT NULL
            ORDER BY source, start_ts DESC
        """)
        rows = cur.fetchall()
        if rows:
            # Print column headers
            colnames = [desc[0] for desc in cur.description]
            print(" | ".join(colnames))
            for row in rows:
                print(" | ".join(str(val) if val is not None else "" for val in row))
        else:
            print("No classes found.")



Most recent class from each source (where start_ts and end_ts are not null):

source | class_name | instructor | location | capacity | spots_available | start_ts | end_ts | status | url
coolcharm | Reformer Mixed Level (All levels) |  | Genk | 5 | 1 | 2025-09-28 10:00:00+00:00 | 2025-09-28 10:55:00+00:00 | Book | 
koepel | Group Class |  | Unknown Location | 4 | 0 | 2025-08-30 15:00:00+00:00 | 2025-08-30 15:45:00+00:00 |  | 
rite | REFORM - ANTWERP | Lieke H. | Riemstraat 13, Antwerp | 19 | 7 | 2025-09-07 18:00:00+00:00 | 2025-09-07 18:50:00+00:00 |  | 
rowreformer | MAT pilates | Feli | Mat studio | 10 | 0 | 2025-09-28 10:00:00+00:00 | 2025-09-28 10:50:00+00:00 |  | 
